In [1]:
import numpy as np
import gdspy
import os
from autograd import numpy as npa

################################################### design parameter #########################
resolution = 50  # 해상도
Mask_thick = 25
design_region_x = round(0.4, 2)
design_region_y = round(7.0 + 2 * Mask_thick / resolution, 2)
design_region_z = round(7.0 + 2 * Mask_thick / resolution, 2)
design_region_resolution = int(resolution)
Nx = int(design_region_resolution * design_region_x) + 1
Ny = int(design_region_resolution * design_region_y) + 1
Nz = int(design_region_resolution * design_region_z) + 1
##############################################################################################

# Numpy 배열 생성 2D 배열을 만듭니다
structure_weight = np.loadtxt('lastdesign.txt')
structure_weight = structure_weight.reshape(Nx, Ny * Nz)[Nx - 1]
data = npa.rot90(structure_weight.reshape(Ny, Nz))  # ZxY

# Check if the cell 'TOP' exists and delete it if it does
if 'TOP' in gdspy.current_library.cells:
    del gdspy.current_library.cells['TOP']

# Create the new cell
cell = gdspy.Cell('TOP')

# Rectangle dimensions
width = 1
height = 1

# Numpy 배열을 기반으로 GDS의 경로(Path) 생성
rectangles = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i, j] == 1:
            x0 = j * width
            y0 = -i * height
            x1 = (j + 1) * width
            y1 = -(i + 1) * height
            rectangle = gdspy.Rectangle((x0, y0), (x1, y1))
            rectangles.append(rectangle)

def is_adjacent(p1, p2, tolerance=1.0):
    """Check if two points are adjacent based on a distance tolerance."""
    return np.linalg.norm(np.array(p1) - np.array(p2)) <= tolerance

def find_shared_edges(rectangles, tolerance=1.0):
    """
    Detect shared edges between adjacent rectangles.
    Returns a set of shared edge points that should not be smoothed.
    """
    shared_edges = set()
    for rect1 in rectangles:
        for rect2 in rectangles:
            if rect1 is not rect2:
                # Check each edge of rect1 and rect2 for overlap
                for p1 in rect1.get_bounding_box():
                    for p2 in rect2.get_bounding_box():
                        if is_adjacent(p1, p2, tolerance):
                            shared_edges.add((tuple(p1), tuple(p2)))
                            shared_edges.add((tuple(p2), tuple(p1)))
    return shared_edges

def chaikin_cutting(points, iterations=1, shared_edges=None):
    """
    Apply Chaikin's Corner Cutting algorithm to smooth corners of a polygon.
    `points`: list of (x, y) tuples representing the polygon.
    `iterations`: number of times the algorithm should be applied.
    `shared_edges`: A set storing shared edge points that should not be smoothed.
    """
    for _ in range(iterations):
        new_points = []
        for i in range(len(points) - 1):
            p0 = tuple(points[i])
            p1 = tuple(points[i + 1])
            # Only smooth if the points are not in shared edges
            if shared_edges and (p0, p1) in shared_edges:
                new_points.append(p0)  # Keep shared edges unchanged
            else:
                # Apply Chaikin's smoothing
                q = (0.75 * p0[0] + 0.25 * p1[0], 0.75 * p0[1] + 0.25 * p1[1])
                r = (0.25 * p0[0] + 0.75 * p1[0], 0.25 * p0[1] + 0.75 * p1[1])
                new_points.extend([q, r])
        new_points.append(tuple(points[-1]))  # Convert the last point to tuple
        points = new_points
    return points

if rectangles:
    expanded_rectangles = [gdspy.offset(rectangle, distance=0.05) for rectangle in rectangles]
    merged_polygon = gdspy.boolean(expanded_rectangles, None, 'or', precision=1e-6, max_points=500)

    if merged_polygon:
        shared_edges = find_shared_edges(rectangles)  # Find shared edges to preserve

        for polygon_boundary in merged_polygon.polygons:
            # Apply smoothing, skipping shared edges
            smoothed_points = chaikin_cutting(polygon_boundary, iterations=2, shared_edges=shared_edges)
            smoothed_polygon = gdspy.Polygon(smoothed_points)
            cell.add(smoothed_polygon)

# Save the GDS file
gdspy.write_gds('output_modified_adj.gds')

# Optionally, save an image of the cell as SVG.
cell.write_svg('output_modified_adj.svg')

# Display all cells using the internal viewer.
gdspy.LayoutViewer()


KeyboardInterrupt: 